<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#获取网页" data-toc-modified-id="获取网页-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>获取网页</a></span></li><li><span><a href="#提取数据" data-toc-modified-id="提取数据-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>提取数据</a></span></li><li><span><a href="#文件保存" data-toc-modified-id="文件保存-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>文件保存</a></span></li><li><span><a href="#分页操作" data-toc-modified-id="分页操作-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>分页操作</a></span></li></ul></div>

# 获取网页
- 使用 try ... except ...，在响应码=200时返回内容
- 添加 headers


In [5]:
import requests
import re
from requests.exceptions import RequestException

# 获取网页
def get_one_page(url): 
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.100 Safari/537.36'
    }
    try:
        html = requests.get(url, headers=headers)
        if html.status_code == 200:
            return html.text
    except RequestException as e:
        print(e.args)
        return None
        

# 提取数据
- 正则一次提取
- 使用生成器整理数据

In [6]:
# 提取，清洗数据
def parse_one_page(html):
#     html = re.sub('\s*', '', html)
    pattern_all =re.compile('<dd>.*?board-index.*?>(.*?)</i>.*?data-src="(.*?)".*?name.*?a.*?>(.*?)</a>.*?star.*?>(.*?)</p>.*?releasetime.*?>(.*?)</p>.*?integer.*?>(.*?)</i>.*?fraction.*?>(.*?)</i>.*?</dd>',
                           re.S) 
    items = re.findall(pattern_all, html)
    for item in items:
            yield {'index': item[0],
                'image': item[1],
                'title': item[2].strip(),
                'actor': item[3].strip()[3:] if len(item[3]) > 3 else '',
                'time': item[4].strip()[5:] if len(item[4]) > 5 else '',
                'score': item[5].strip() + item[6].strip()}

# 文件保存
从 python → 磁盘需要将数据对象序列化，针对 dict ，用 json 序列化的方法比较好


通过 json.dumps() 实现 dict 的**序列化**

指定 ensure_ascii = False，保证输出结果是中文形式而不是 Unicode 编码

In [13]:
import json

def write_to_file(content):
    with open('result.txt', 'a', encoding='utf8') as f:
        print(json.dumps(content))    # 查看结果
        f.write(json.dumps(content, ensure_ascii=False)+'\n')    # 写入

# 分页操作
1. 修改 URL 
2. 休息，防反爬

In [14]:
import time

if __name__ == '__main__':
    for i in range(10):
        url = 'https://maoyan.com/board/4?offset=' + str(i*10)
        html = get_one_page(url) 
        items = parse_one_page(html)
        for item in items:
            write_to_file(item)
        time.sleep(1)    # 防反爬

{'index': '1', 'image': 'https://p1.meituan.net/movie/20803f59291c47e1e116c11963ce019e68711.jpg@160w_220h_1e_1c', 'title': '霸王别姬', 'actor': '张国荣,张丰毅,巩俐', 'time': '1993-01-01', 'score': '9.5'}
{'index': '2', 'image': 'https://p0.meituan.net/movie/283292171619cdfd5b240c8fd093f1eb255670.jpg@160w_220h_1e_1c', 'title': '肖申克的救赎', 'actor': '蒂姆·罗宾斯,摩根·弗里曼,鲍勃·冈顿', 'time': '1994-09-10(加拿大)', 'score': '9.5'}
{'index': '3', 'image': 'https://p0.meituan.net/movie/289f98ceaa8a0ae737d3dc01cd05ab052213631.jpg@160w_220h_1e_1c', 'title': '罗马假日', 'actor': '格利高里·派克,奥黛丽·赫本,埃迪·艾伯特', 'time': '1953-09-02(美国)', 'score': '9.1'}
{'index': '4', 'image': 'https://p1.meituan.net/movie/6bea9af4524dfbd0b668eaa7e187c3df767253.jpg@160w_220h_1e_1c', 'title': '这个杀手不太冷', 'actor': '让·雷诺,加里·奥德曼,娜塔莉·波特曼', 'time': '1994-09-14(法国)', 'score': '9.5'}
{'index': '5', 'image': 'https://p1.meituan.net/movie/b607fba7513e7f15eab170aac1e1400d878112.jpg@160w_220h_1e_1c', 'title': '泰坦尼克号', 'actor': '莱昂纳多·迪卡普里奥,凯特·温丝莱特,比利·赞恩', 'time': '19